In [1]:
import re
import sys
import io
import preprocessor as p

# input and output files
infile = "/users/kumaraprasannajayaraju/Downloads/NLP_Final_Project/Method 1/data/Tweets2Vec_DA.rtf"
outfile = "/users/kumaraprasannajayaraju/Downloads/NLP_Final_Project/Method 1/data/life_t2v_ds_en_op.txt"

regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)+' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    tokens = [token.lower() for token in tokens]

    html_regex = re.compile('<[^>]+>')
    tokens = [token for token in tokens if not html_regex.match(token)]

    mention_regex = re.compile('(?:@[\w_]+)')
    tokens = ['@mention' if mention_regex.match(token) else token for token in tokens]

    url_regex = re.compile('http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
    tokens = ['@url' if url_regex.match(token) else token for token in tokens]

    hashtag_regex = re.compile("(?:\#+[\w_]+[\w\'_\-]*[\w_]+)")
    tokens = ['@hash' if hashtag_regex.match(token) else token for token in tokens]    

    f = p.clean(' '.join([t for t in tokens if t]).replace('rt','')
                   .replace(':','').replace('...','')
                   .replace('@mention', '').replace('@url', '').replace('@hash', ''))
    
    return f


with io.open(outfile, 'w') as tweet_processed_text, io.open(infile, 'r') as fin:
    for line in fin:
        tweet_processed_text.write(preprocess(line)+'\n')